## Presentation Types with Hidden Types
## Batch Output to CSV in Sets

In [2]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy as np
import itertools
from itertools import combinations
import networkx as nx
from community import community_louvain
from copy import deepcopy
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


#### The following are special functions used by the classifier.  Don't change them.

## Load one Piece Here

* Note that you can load from CRIM, or put a file in the **Music_Files** folder in the Notebook.

In [3]:
git_prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'

# just add the CRIM Piece ID here
mei_file = 'CRIM_Model_0001.mei'


url = git_prefix + mei_file
# piece = importScore('Music_Files/Senfl_Ave_forCRIM.mei_msg.mei')
piece = importScore(url)
# piece = importScore('Music_Files/CRIM_Mass_0007_4.mei')

print(piece.metadata)

Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0001.mei
{'title': 'Veni speciosam', 'composer': 'Lupi, Johannes'}


## Run the Classifier Here

- set the length of the soggetti with `melodic_ngram_length`
- set the maximum difference between similar soggetti with `edit_distance_threshold`
- for chromatic vs diatonic, compound, and directed data in soggetti, see `interval_settings`
- to include all the hidden PENs and IDS (those found within longer Fugas, use `include_hidden_types == True`.  
- for faster (and simpler) listing of points of imitation without hidden forms, use `include_hidden_types == False`



### Run Classifier on Several Pieces at Once

Results are combined into a single dataframe

In [4]:


git_prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'

# piece = importScore('Music_Files/CRIM_Mass_0007_4.mei')
piece_list =  ['CRIM_Mass_0005_3.mei',
             'CRIM_Mass_0005_4.mei',
             'CRIM_Mass_0005_5.mei',
             'CRIM_Model_0001.mei',
             'CRIM_Mass_0002_1.mei',
             'CRIM_Mass_0002_2.mei',
             'CRIM_Mass_0002_3.mei',
             'CRIM_Mass_0002_4.mei',
             'CRIM_Mass_0002_5.mei',
             'CRIM_Model_0015.mei',
             'CRIM_Mass_0013_1.mei',
             'CRIM_Mass_0013_2.mei',
             'CRIM_Mass_0013_3.mei',
             'CRIM_Mass_0013_4.mei',
             'CRIM_Mass_0013_5.mei',
             'CRIM_Model_0019.mei',
             'CRIM_Mass_0019_1.mei',
             'CRIM_Mass_0019_2.mei',
             'CRIM_Mass_0019_3.mei',
             'CRIM_Mass_0019_4.mei',
             'CRIM_Mass_0019_5.mei']


In [5]:
git_prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'

piece_list = ['CRIM_Model_0019.mei',
             'CRIM_Mass_0019_1.mei',
             'CRIM_Mass_0019_2.mei',
             'CRIM_Mass_0019_3.mei',
             'CRIM_Mass_0019_4.mei',
             'CRIM_Mass_0019_5.mei']

In [37]:
mass_sets = [('CRIM_Mass_0019',
('CRIM_Model_0019.mei',
 'CRIM_Mass_0019_1.mei',
 'CRIM_Mass_0019_2.mei',
 'CRIM_Mass_0019_3.mei',
 'CRIM_Mass_0019_4.mei',
'CRIM_Mass_0019_5.mei'))]

git_prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'

include_hidden_types = False
combine_unisons = True
melodic_ngram_length = 4
edit_distance_threshold = 0

for mass in mass_sets:
    final = pd.DataFrame()
    corpus_set = []
    csv_name = 'saved_csv/' + mass_sets[0][0] + '_' + 'P_Types'
    file_names = mass[1]
    for file in file_names:
        file_url = git_prefix + file 
        url = git_prefix + file


        piece = importScore(url)   
        nr = piece.getNoteRest(combineUnisons=combine_unisons)
        dur = piece.getDuration(df=nr)
        mel = piece.getMelodic(df=nr, kind='d', end=False)
        dur_ng = piece.getNgrams(df=dur, n=melodic_ngram_length)
        mel_ng = piece.getNgrams(df=mel, n=melodic_ngram_length)
        entries = piece.getEntries(mel_ng)

        output = classify_entries_as_presentation_types(piece, nr, dur_ng, entries, edit_distance_threshold, include_hidden_types)


        final = final.append(output, ignore_index=True)

final.to_csv(csv_name)

Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.


In [36]:
final

,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
0,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0, 11/3.0, 14/1.0, 1...","[P-5, P-4, P-5, P8, P5, P-8]","[0.0, 8.0, 40.0, 48.0, 84.0, 104.0, 128.0]","[-3, 3, 2, -2]","[8.0, 32.0, 8.0, 36.0, 20.0, 24.0]","[Cantus, Altus, Tenor, Bassus, Altus, Cantus, ...",FUGA,7,False
1,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,150.0,"[19/4.0, 21/4.0, 22/4.0, 25/4.0, 27/4.0, 29/4....","[P8, P-5, P-4, P-5, P8, P8, P-11]","[150.0, 166.0, 174.0, 198.0, 214.0, 230.0, 250...","[-3, 2, 2, -2]","[16.0, 8.0, 24.0, 16.0, 16.0, 20.0, 24.0]","[Tenor, Cantus, Altus, Tenor, Bassus, Altus, C...",FUGA,8,False
2,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,298.0,"[38/2.0, 38/4.0, 42/2.0, 44/1.0, 46/2.0, 48/4.0]","[P-5, P-5, P8, P8, P-11]","[298.0, 302.0, 330.0, 344.0, 362.0, 382.0]","[-3, 3, -2, -2]","[4.0, 28.0, 14.0, 18.0, 20.0]","[Altus, Tenor, Bassus, Altus, Cantus, Bassus]",FUGA,6,False
3,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,394.0,"[50/2.0, 51/4.0, 53/2.0, 55/2.0, 56/4.0, 59/1....","[P5, P-8, P4, P5, P-8, P8, P-4, P-5]","[394.0, 406.0, 418.0, 434.0, 446.0, 464.0, 486...","[-2, 2, 3, -2]","[12.0, 12.0, 16.0, 12.0, 18.0, 22.0, 4.0, 16.0]","[Altus, Cantus, Bassus, Tenor, Altus, Bassus, ...",FUGA,9,False
4,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0]","[P-5, P-4, P-5]","[0.0, 8.0, 40.0, 48.0]","[-3, 3, 2, -2]","[8.0, 32.0, 8.0]","[Cantus, Altus, Tenor, Bassus]",ID,4,False
5,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,78.0,"[10/4.0, 12/2.0, 14/2.0, 15/3.0]","[P-4, P-5, P-4]","[78.0, 90.0, 106.0, 116.0]","[2, 2, 2, 2]","[12.0, 16.0, 10.0]","[Cantus, Altus, Tenor, Bassus]",FUGA,4,False
6,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,160.0,"[20/1.0, 24/1.0, 30/2.0, 33/1.0, 35/3.0, 39/3.0]","[P-8, P1, P4, P-8, P5]","[160.0, 192.0, 242.0, 264.0, 284.0, 316.0]","[-3, 2, 2, -2]","[32.0, 50.0, 22.0, 20.0, 32.0]","[Cantus, Tenor, Bassus, Altus, Bassus, Bassus]",FUGA,6,False
7,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,164.0,"[20/3.0, 24/3.0, 27/2.0, 27/4.0, 30/4.0, 36/1....","[P-8, P8, P-8, P5, P4, P5, M-6]","[164.0, 196.0, 218.0, 222.0, 246.0, 288.0, 322...","[2, 2, -2, -3]","[32.0, 22.0, 4.0, 24.0, 42.0, 34.0, 4.0]","[Altus, Bassus, Cantus, Tenor, Tenor, Altus, C...",FUGA,8,False
8,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,372.0,"[45/3.0, 47/3.0, 49/3.0, 51/4.0, 53/3.0]","[P8, P-8, P8, P-12]","[372.0, 388.0, 404.0, 422.0, 436.0]","[-3, 3, -2, -2]","[16.0, 16.0, 18.0, 14.0]","[Tenor, Cantus, Bassus, Altus, Bassus]",FUGA,5,False
9,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,458.0,"[56/2.0, 58/4.0, 61/3.0, 63/2.0]","[P-8, P-5, P12]","[458.0, 478.0, 500.0, 514.0]","[-2, 2, 3, -2]","[20.0, 22.0, 14.0]","[Cantus, Tenor, Bassus, Cantus]",FUGA,4,False
